In [ ]:
import os
from os import path
import json

## Create a web service for inference using the Tensorflow model without Azure ML
With Azure Machine Learning Service, you don't have to do this any more. But it does tell you how Azure ML deploys a web service from a ML model for you. Also refer to [this article](https://liupeirong.github.io/amlDockerImage/) about what's inside the Docker image built by Azure ML.

In [ ]:
%%writefile kerasWebApp/keras_score.py

import numpy as np
from numpy import array
from PIL import Image
import requests
from io import BytesIO
from keras.models import model_from_json
import utils

def init():
    global model, unique_labels, target_imagesize
    json_file = open('kerasModel.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights('kerasModel.h5')
    target_imagesize = 128
    unique_labels = (['axes', 'boots', 'carabiners', 'crampons', 'gloves', 'harnesses', 'helmets', 'pulleys', 'rope', 'tents'])

def index_to_category(index):
    return unique_labels[index]

def run(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img_arr = utils.processImg(img, target_imagesize)
    predictions = model.predict_classes(array([img_arr]))
    
    return index_to_category(predictions[0])

def main():
    # Test the init and run functions using test data
    test_url = "/url/to/image"
    init()
    category = run(test_url)
    print(category)

if __name__ == "__main__":
    main()


## Module for Flask web app

In [ ]:
%%writefile kerasWebApp/app.py
from flask import Flask, request
import json
import keras
from keras_score import *

app = Flask(__name__)

@app.route('/score', methods = ['POST'])
def scoreKeras():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    input = request.json['url']
    response = run(input)
    dict = {}
    dict['result'] = response
    return json.dumps(dict)


@app.route("/")
def healthy():
    return "Healthy"


# Keras Version
@app.route('/version', methods = ['GET'])
def version_request():
    return keras.__version__


if __name__ == "__main__":
    app.run(host='0.0.0.0') # Ignore, Development server


In [ ]:
%%writefile kerasWebApp/wsgi.py
import sys
sys.path.append('/code/') # FIXME: This is horrible
from app import app as application
from keras_score import *

def create():
    print("Initialising")
    init()
    application.run(host='127.0.0.1', port=5000)

In [ ]:
%%writefile kerasWebApp/requirements.txt
pillow
Flask==0.11.1
requests==2.12.3
gunicorn==19.6.0
json-logging-py==0.2
keras==2.1.4
h5py
scikit-learn
tensorflow==1.6.0

In [ ]:
%%writefile kerasWebApp/nginx/app
server {
    listen 88;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

In [ ]:
%%writefile kerasWebApp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

In [ ]:
%%writefile kerasWebApp/kill_supervisor.py
import sys
import os
import signal


def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()

In [ ]:
%%writefile kerasWebApp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

In [ ]:
%%writefile kerasWebApp/dockerfile

FROM ubuntu:16.04

RUN mkdir /code
WORKDIR /code
ADD . /code/
ADD etc /etc

RUN apt-get update && apt-get install -y --no-install-recommends \
        openmpi-bin \
        python3-pip \ 
        python3-dev \ 
        python3-setuptools \
        supervisor \
        nginx && \
    ln -s /usr/bin/python3 python && \
    pip3 install --upgrade pip && \
    rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ && \
    pip install -r /code/requirements.txt

EXPOSE 88
CMD ["supervisord", "-c", "/etc/supervisord.conf"]


## Test locally
```bash
curl --request POST --header "Content-Type: application/json" --data '{"url": "/url/to/image"}' http://0.0.0.0:88/score
```

## Deploy to Azure Kubernetes Service

1. Push the docker image to ACR
```bash
az acr login -n {my_acr}
docker image tag {my_image} {my_acr}.azurecr.io/{my_image}
docker push {my_acr}.azurecr.io/{my_image}
# test the image
docker run --rm -p 88:88 {my_acr}.azurecr.io/{my_image}
# run the curl command above to test the web service
```
2. Create AKS in the portal or using Azure Cli
  - To connect to AKS, make sure you have the latest Azure Cli. On DSVM, you may need to do the following - 
```bash
# dollar sign crashes Jupyter notebook, when you run the following command, replace with real dollar sign
pip uninstall azure-cli
AZ_REPO=dollar_sign(lsb_release -cs)
echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ dollar_sign{AZ_REPO} main" | sudo tee /etc/apt/sources.list.d/azure-cli.list
curl -L https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
sudo apt-get install apt-transport-https
sudo apt-get update && sudo apt-get install azure-cli
whereis az
# if az is still the old version
sudo cp /usr/bin/az /opt/az/bin/az
sudo mv /opt/az/bin/az.bat /tmp
ln -s /usr/bin/az /anaconda/envs/py35/bin/az
```
3. Connect Kubenetes Client
```bash
az aks get-credentials --resource-group {my_rg} --name {my_aks}
# install kubenetes client
sudo chmod 777 /usr/local/bin
az aks install-cli
kubectl cluster-info # should return AKS info
kubectl get nodes # should return AKS nodes
kubectl proxy # on DSVM, it says 8001 is already in use, on Windows, it only returns API interface not dashboard
# bring up Kubenetes dashboard, this works on Windows, doesn't work on DSVM
az aks browse --resource-group {my_rg} --name {my_aks}
```
4. Grant AKS access to ACR
```bash
# get the service principal that AKS uses to manage cluster resources
az aks show --resource-group {my_rg} --name {my_aks} --query "servicePrincipalProfile.clientId" --output tsv
# grant AKS service principal read access to ACR
az acr show --name {my_acr} --resource-group {my_rg} --query "id" --output tsv
az role assignment create --assignee {service_principal_client_id} --role Reader --scope {acr_id} 
```
5. Deploy the web service (in docker image) to AKS
  - Go to the Kubenetes dashboard, +Create a deployment, give it a name, the ACR image tag, add an external service which maps a port to port 88 on the container. Deploy. 
  - Run ```kubectl get pods```, ```kubectl get svc``` to see the progress and the service endpoint
  - Test against the external IP of the service endpoint
  - To test a simple app, run ```kubectl run kubernetes-bootcamp --image=gcr.io/google-samples/kubernetes-bootcamp:v1 --port=8080```